# YOLOv5 Notebook Demo

## Clone repo

In [ ]:
!git clone https://github.com/aman-cc/YOLOv5.git -b pascal_data --depth 1
%cd YOLOv5

## Install needed requirements
Uncomment below if not in colab

In [ ]:
# !pip install -r requirements.txt
# !pip install gdown matplotlib

In [ ]:
import os
import torch
import yaml

import matplotlib as plt

import yolo
from yolov5 import YOLOv5

## Download PASCAL VOC Dataset for training

In [ ]:
!gdown 'https://drive.google.com/uc?id=10NFCj8VQ4h8TFZGb5IEmyk1ePCLAUU-e'
!tar xf pascal_data.tar.gz

## Load Dataloader

In [ ]:
with open("./config.yaml", "r") as stream:
    args = yaml.safe_load(stream)

yolo_obj = YOLOv5(args)

if not os.path.isdir(args["DATA_DIR"]):
    raise Exception(
        "COCO data not download. Please download COCO using './download_coco.sh'"
    )
splits = ("train2017", "val2017")
file_roots = [os.path.join(yolo_obj.data_dir, "images", x) for x in splits]
ann_files = [
    os.path.join(yolo_obj.data_dir, "annotations/instances_{}.json".format(x))
    for x in splits
]
if not os.path.isdir(args["EXPT_DIR"]):
    os.makedirs(args["EXPT_DIR"], exist_ok=True)

transforms = yolo.RandomAffine((0, 0), (0.1, 0.1), (0.9, 1.1), (0, 0, 0, 0))
dataset_train = yolo.datasets(
    yolo_obj.dataset, file_roots[0], ann_files[0], train=True
)
dataset_test = yolo.datasets(
    yolo_obj.dataset, file_roots[1], ann_files[1], train=True
)  # set train=True for eval
if len(dataset_train) < yolo_obj.batch_size:
    raise Exception(
        f"Very low number of samples. Available samples: {len(dataset_train)} | Batch size: {yolo_obj.batch_size}"
    )

sampler_train = torch.utils.data.RandomSampler(dataset_train)
sampler_test = torch.utils.data.SequentialSampler(dataset_test)

batch_sampler_train = yolo.GroupedBatchSampler(
    sampler_train, dataset_train.aspect_ratios, yolo_obj.batch_size, drop_last=True
)
batch_sampler_test = yolo.GroupedBatchSampler(
    sampler_test, dataset_test.aspect_ratios, yolo_obj.batch_size
)

num_workers = min(
    os.cpu_count() // 2, 8, yolo_obj.batch_size if yolo_obj.batch_size > 1 else 0
)
device = torch.device(
    "cuda" if torch.cuda.is_available() and yolo_obj.use_cuda else "cpu"
)
cuda = device.type == "cuda"
data_loader_train = torch.utils.data.DataLoader(
    dataset_train,
    batch_sampler=batch_sampler_train,
    num_workers=num_workers,
    collate_fn=yolo.collate_wrapper,
    pin_memory=cuda,
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_sampler=batch_sampler_test,
    num_workers=num_workers,
    collate_fn=yolo.collate_wrapper,
    pin_memory=cuda,
)

d_train = yolo.DataPrefetcher(data_loader_train) if cuda else data_loader_train
d_test = yolo.DataPrefetcher(data_loader_test) if cuda else data_loader_test

num_classes = len(dataset_train.classes)
warmup_iters = max(1000, 3 * len(dataset_train))
save_path = os.path.join(args["EXPT_DIR"], "ckpt")
yolo_obj.load_model(num_classes, warmup_iters, device)
yolo_obj.load_weights("yolov5s_official_2cf45318.pth", device, pretrained=True)
yolo_obj.train(d_train, d_test, save_path, device)

## View train loss and mAP

In [ ]:
with open('train_monitoring.yaml', 'r') as f:
    train_metrics = yaml.safe_load(f)
loss_vals = train_metrics['loss']
mAP_vals = train_metrics['mAP']
epochs = range(1, len(loss_vals)+1)

f, ax = plt.pyplot.subplots(nrows=1,ncols=2,figsize=(18,5))
    
ax[0].plot(epochs, loss_vals,  marker='o', label='Training Loss')
ax[0].set_title('Training Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].legend(loc='best')
ax[0].grid(True)

ax[1].plot(epochs, mAP_vals, marker='o', label='Training mAP')
ax[1].set_title('Training mAP')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('mAP')
ax[1].legend(loc='best')
ax[1].grid(True)

f.savefig('train_metrics.png', format='png')

## Evaluate mAP on the test set

In [ ]:
yolo_obj.evaluate(data_loader_test, device)

# Sample inference

In [ ]:
results = yolo_obj.infer(data_loader_test, device)

In [ ]:
num_preds = 5
for i, data in enumerate(data_loader_test):
    images = data.images
    targets = data.targets
    
    with torch.no_grad():
        results, losses = yolo_obj.model(images)
        
    # images' saving names
    save = [os.path.join(args["EXPT_DIR"], f"{tgt['image_id'].item()}.jpg") for tgt in targets]
    yolo.show(images, results, dataset_test.classes, save)

    if i > num_preds:
        break